In [1]:
import awswrangler as wr
from datetime import datetime, timedelta
import numpy as np
import os
import sys

In [2]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/lambda") 

In [6]:
import app

In [ ]:
months = app.lambda_handler({
    "function": {
        "name": "get_months",
        "parameters": {
        }
    }
}, None)
months[0]

'202207'

In [ ]:
# batches = app.lambda_handler({
#     "function": {
#         "name": "find_missing_raw_ticks",
#         "parameters": {
#             "month": months[-1],
#             "batch_size": 12,
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL"
#         }
#     }
# }, None)

In [ ]:
# app.lambda_handler({
#     "function": {
#         "name": "save_missing_raw_ticks",
#         "parameters": {
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL",
#             "batch_id": batches[0][0],
#         }
#     }
# }, None)

In [ ]:
# batches = app.lambda_handler({
#     "function": {
#         "name": "find_missing_clean_ticks",
#         "parameters": {
#             "month": months[0],
#             "batch_size": 3,
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL"
#         }
#     }
# }, None)

In [ ]:
# app.lambda_handler({
#     "function": {
#         "name": "save_missing_clean_ticks",
#         "parameters": {
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL",
#             "batch_id": batches[0][0],
#         }
#     }
# }, None)

In [ ]:
# batches = app.lambda_handler({
#     "function": {
#         "name": "find_missing_candles_from_ticks",
#         "parameters": {
#             "month": months[0],
#             "batch_size": 3,
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL"
#         }
#     }
# }, None)

In [ ]:
# app.lambda_handler({
#     "function": {
#         "name": "save_missing_candles_from_ticks",
#         "parameters": {
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL",
#             "batch_id": batches[0][0],
#         }
#     }
# }, None)

In [ ]:
# batches = app.lambda_handler({
#     "function": {
#         "name": "find_missing_candles",
#         "parameters": {
#             "month": months[0],
#             "batch_size": 3,
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL",
#             "name_dataset_in": "from_ticks",
#             "name_dataset_out": "ffill",
#             "function_names": ["ffill"],
#         }
#     }
# }, None)

In [7]:
# app.lambda_handler({
#     "function": {
#         "name": "save_missing_candles",
#         "parameters": {
#             "dynamo_table": "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL",
#             "batch_id": batches[0][0],
#         }
#     }
# }, None)

True

In [ ]:
# item_ids = (
#     wr.dynamodb
#     .get_table("mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL")
#     .scan(AttributesToGet=["id",])["Items"]
# )
# wr.dynamodb.delete_items(item_ids, "mercury-pipeline-metaapi-ControllerTable-15QM09J0Q9TVL")

In [ ]:
assert False

AssertionError: 

In [ ]:
from src.config import *
from src.io import *
from src.utils import *

In [ ]:
result = lambda_handler({"function": "setup_save_raw_ticks", "stop":False}, None)
result[0][0]

In [ ]:
# await process_batch_async(save_raw_ticks, batch_id=result[0][0])

In [ ]:
# lambda_handler({"function": "save_raw_ticks", "batch_id":result[0][0]}, None)

In [ ]:
result = lambda_handler({"function": "setup_save_clean_ticks"},  None)
result[0][0]

In [ ]:
# lambda_handler({"function": "save_clean_ticks", "batch_id":result[0][0]}, None)

In [ ]:
result = lambda_handler({"function": "setup_save_candles_from_ticks"},  None)
result[0]

In [ ]:
# lambda_handler({"function": "save_candles_from_ticks", "batch_id":result[0][0]},  None)

In [ ]:
result = lambda_handler(
    {
        "function": "setup_save_candles_transformed",
        "name_dataset_in": "from_ticks",
        "name_dataset_out": "ffill",
        "function_names": ["ffill"],
    },
    None
)
result[0][0]

In [ ]:
%%time
lambda_handler({"function": "save_candles_transformed", "batch_id":result[0].pop()},  None)

In [ ]:
table = wr.dynamodb.get_table(TABLE_PIPELINE)
batch = table.get_item(Key={"id":0})["Item"]

In [ ]:
batch["batch"][0]

In [ ]:
parameters_in = get_parameters_from_key(batch["batch"][0]["prefix_in"])
parameters_lookback = get_parameters_from_key(batch["batch"][0]["prefix_lookback"])

In [ ]:
name_dataset_in = batch["batch"][0]["prefix_in"].split("/candles/")[-1].split("/")[0]
name_dataset_in

In [ ]:
parameters_in

In [ ]:
parameters_lookback

In [ ]:
datetime_in = datetime.strptime(
    parameters_in["date"] + parameters_in["hour"], 
    "%Y%m%d%H"
)
datetime_lookback = datetime.strptime(
    parameters_lookback["date"] + parameters_lookback["hour"], 
    "%Y%m%d%H"
)
datetime_in, datetime_lookback

In [ ]:
n_hours = int((datetime_in - datetime_lookback).total_seconds() / 3600) + 1
n_hours

In [ ]:
hours = [datetime_lookback + timedelta(hours=i) for i in range(n_hours)]
hours

In [ ]:
keys = [
    f"s3://{BUCKET}/{DIR_CANDLES_ROOT}/" + "/".join([
        name_dataset_in,
        f"symbol={parameters_in['symbol']}",
        f"frequency={parameters_in['frequency']}",
        f"year={dt.strftime('%Y')}",
        f"month={dt.strftime('%m')}",
        f"date={dt.strftime('%Y%m%d')}",
        f"hour={dt.strftime('%H')}",
        ""
    ]) for dt in hours
]
keys

In [ ]:
l = []
for key in keys:
    l.append(wr.s3.read_parquet(key))

In [ ]:
pd.concat(l)

In [ ]:
wr.s3.list_objects("s3://datalake.dgriffiths.io/projects/mercury/data/clean/candles/from_ticks/symbol=EURUSD/frequency=1H/year=2022/month=07/date=20220708/hour=21/")

In [ ]:
wr.s3.does_object_exist("s3://datalake.dgriffiths.io/projects/mercury/data/clean/candles/from_ticks/symbol=EURUSD/frequency=1H/year=2022/month=07/date=20220707/hour=07")

In [ ]:
wr.s3.list_objects("s3://datalake.dgriffiths.io/projects/mercury/data/clean/candles/from_ticks/symbol=EURUSD/frequency=1H/year=2022/month=07/date=20220707/hour=07") != []